In [2]:
!pip install numpy



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
!pip install tensorflow

  Using cached tensorflow-2.18.0-cp310-cp310-win_amd64.whl.metadata (3.3 kB)
  Using cached tensorflow_intel-2.18.0-cp310-cp310-win_amd64.whl.metadata (4.9 kB)
Using cached tensorflow-2.18.0-cp310-cp310-win_amd64.whl (7.5 kB)
Using cached tensorflow_intel-2.18.0-cp310-cp310-win_amd64.whl (390.0 MB)


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\user\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python310\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\ext\\filters\\client_channel\\lb_policy\\grpclb\\client_load_reporting_filter.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import sqlite3
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import numpy as np
from tensorflow.keras.models import load_model


ModuleNotFoundError: No module named 'tensorflow'

In [7]:
# Function to set up the database (if not already set up)
def setup_database():
    conn = sqlite3.connect('users.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT NOT NULL UNIQUE,
            password TEXT NOT NULL
        )
    ''')
    conn.commit()
    conn.close()

In [8]:
# Load the pre-trained VGG16 model (fine-tuned for fruit classification)
model = load_model(r'D:\Project\Danial\Final\model\99 percent\gg16_fruit_classification_model.h5')


In [1]:
# Function to handle user login
def login():
    username = username_entry.get()
    password = password_entry.get()
    
    conn = sqlite3.connect('users.db')
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM users WHERE username=? AND password=?', (username, password))
    user = cursor.fetchone()
    conn.close()

    if user:
        messagebox.showinfo("Login", "Login Successful!")
        open_image_upload_window()
    else:
        messagebox.showerror("Login", "Invalid username or password.")

        # Function to open the image upload window
def open_image_upload_window():
    login_window.destroy()
    upload_window = tk.Tk()
    upload_window.title("FruitVision: Upload Image")
    upload_window.geometry("800x600")
    upload_window.configure(bg="lightblue")

    def upload_image():
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
        if file_path:
            try:
                img = Image.open(file_path)
                img = img.convert('RGB')
                img = img.resize((224, 224))  # Resize the image to 100x100 (same as VGG16 input size)
                img_array = np.array(img) / 255.0  # Normalize the image
                img_array = np.expand_dims(img_array, axis=0)  # Prepare for model input

                # Classify the image using the loaded VGG16 model
                prediction = model.predict(img_array)
                class_idx = np.argmax(prediction, axis=1)[0]

                class_names = [
                'Fresh Apple', 'Fresh Banana',
                'Fresh Orange', 'Rotten Apple',
                'Rotten Banana', 'Rotten Orange'
                ]
                result_label.config(text=f"This fruit is: {class_names[class_idx]}", fg="yellow", bg="red")
                
                
                # Display the uploaded image
                img = img.resize((250, 250), Image.LANCZOS)
                img = ImageTk.PhotoImage(img)
                image_label.config(image=img)
                image_label.image = img  # Prevent garbage collection
            except Exception as e:
                print(f"Error: {e}")  # Print the full error to the console
                messagebox.showerror("Error", f"Failed to load or classify image: {e}")
        else:
            messagebox.showwarning("No file selected", "Please select an image file.")

    upload_btn = tk.Button(upload_window, text="Upload Image", command=upload_image)
    upload_btn.pack(pady=20)

    image_label = tk.Label(upload_window)
    image_label.pack()

    result_label = tk.Label(upload_window, text="", font=("Arial", 16))
    result_label.pack(pady=20)

    upload_window.mainloop()

# Function to open the "Create Account" window
def open_create_account_window():
    create_account_window = tk.Tk()
    create_account_window.title("FruitVision: Create Account")
    create_account_window.geometry("800x600")

    def create_account():
        username = new_username_entry.get()
        password = new_password_entry.get()

        conn = sqlite3.connect('users.db')
        cursor = conn.cursor()
        try:
            cursor.execute('INSERT INTO users (username, password) VALUES (?, ?)', (username, password))
            conn.commit()
            messagebox.showinfo("Account Creation", "Account created successfully!")
            create_account_window.destroy()
        except sqlite3.IntegrityError:
            messagebox.showerror("Error", "Username already exists. Please choose another username.")
        finally:
            conn.close()

    tk.Label(create_account_window, text="Username").pack(pady=5)
    new_username_entry = tk.Entry(create_account_window)
    new_username_entry.pack(pady=5)

    tk.Label(create_account_window, text="Password").pack(pady=5)
    new_password_entry = tk.Entry(create_account_window, show='*')
    new_password_entry.pack(pady=5)

    create_btn = tk.Button(create_account_window, text="Create Account", command=create_account)
    create_btn.pack(pady=20)

    create_account_window.mainloop()

# Main Application Window - Login
setup_database()  # Ensure the database is set up
login_window = tk.Tk()
login_window.title("FruitVision: Login")
login_window.geometry("900x600")
login_window.configure(bg="lightblue")

tk.Label(login_window, text="Welcome to Fruit Classification", bg="red", fg="yellow", font=('Helvetica', 26)).pack(pady=10)

# Add the image to the front page
try:
    front_image = Image.open("D://Fruits.jfif")  # Replace with your image file path
    front_image = front_image.resize((200, 200), Image.LANCZOS)
    front_photo = ImageTk.PhotoImage(front_image)
    image_label = tk.Label(login_window, image=front_photo)
    image_label.pack(padx=5, pady=5)
except Exception as e:
    messagebox.showerror("Error", f"Failed to load image: {e}")

tk.Label(login_window, text="Please enter your user name and password", bg="purple", fg="white", font=('Helvetica', 8)).pack(pady=10)
tk.Label(login_window, text="Username", bg="lightgreen", font=('Helvetica')).pack(pady=5)
username_entry = tk.Entry(login_window)
username_entry.pack(pady=5)

tk.Label(login_window, text="Password", bg="lightgreen", font=('Helvetica')).pack(pady=5)
password_entry = tk.Entry(login_window, show='*')
password_entry.pack(pady=5)

login_btn = tk.Button(login_window, text="Login", bg="#4CAF50", fg="#ffffff", font=("Arial", 12, "bold"), bd=0, relief="flat", command=login)
login_btn.pack(padx=10, pady=5)

create_account_btn = tk.Button(login_window, text="Create Account", bg="sky blue", fg="black", font=("Arial", 11, "bold"), bd=0, relief="flat", command=open_create_account_window)
create_account_btn.pack(pady=10)

tk.Label(login_window, text="Click on Create Account button if you don't have an Account", bg="grey", font=('Helvetica', 8)).pack(pady=10)

login_window.mainloop()

NameError: name 'setup_database' is not defined